# LSTM Baseline Model Testing

## 0 Imports & Constants

In [ ]:
import sys
import os

# Füge das übergeordnete Verzeichnis zu sys.path hinzu
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy as dc
from sklearn.preprocessing import MinMaxScaler
import itertools
from tqdm import tqdm
import json
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from utils.utils import load_time_series, add_lagged_data, scale_data, train_test_split_to_tensor, inverse_scale_data
from baseline_model import TimeSeriesDataset

from baseline_model.LSTM import LSTM, train_model

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
DATA_FOLDER = Path("../data")
MULTIVARIATE_DATA_FOLDER = DATA_FOLDER / "multivariate"
UNIVARIATE_DATA_FOLDER = DATA_FOLDER / "univariate"
BENCHMARK = False

## 1 Data

### Data Loading

In [ ]:
# Load data from csv
# -> convert Date column to datetime
data = load_time_series(f'{UNIVARIATE_DATA_FOLDER}/NVDA_open_high_low_close_adjClose_volume_99_24.csv')

## 2 Benchmark Loop

In [ ]:
hyperparameters = {
    'lag': [7, 14, 21],
    'lr': [0.002, 0.001],
    'hidden_size': [2, 4, 6, 8, 12],
    'num_layers': [1, 2],
    'batch_size': [8, 16, 32, 64],
}

In [ ]:
# get all combinations of hyperparameters
keys, values = zip(*hyperparameters.items())
possible_hyperparameters = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
possible_features = [
    ['Close', 'Volume'],
    ['Close', 'Open', 'Volume'],
    ['Close', 'Open', 'High', 'Low', 'Volume']
]

In [ ]:
RESULTS = {}
for features in possible_features:

    for hyperparameters in tqdm(possible_hyperparameters):
        
        print('*'*50)
        print('*'*50)
        print('STARTING NEW BENCHMARK RUN:')
        print(f"Features: {features}")
        print(f"Hyperparameters: {hyperparameters}")
        print('*'*50)
        print('*'*50)

        ### Select features
        features_incl_date = features+['Date']
        data_only_features = data[features_incl_date]
        
        ### Data Preprocessing
        data_lagged = add_lagged_data(data_only_features, hyperparameters['lag'], features)
        data_lagged_scaled, scaler_close = scale_data(data_lagged)
        X_train, y_train, X_test, y_test = train_test_split_to_tensor(data_lagged_scaled)

        ### Create datasets and DataLoaders
        train_dataset = TimeSeriesDataset(X_train, y_train)
        test_dataset = TimeSeriesDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=hyperparameters['batch_size'], shuffle=False)

        ### Train model
        validation_losses = [] # reset validation losses
        for i in range(2): # train 2 times because sometimes the model converges to a local minimum
            ### Instantiate model
            model = LSTM(
                device=device,
                input_size=len(features),
                hidden_size=hyperparameters['hidden_size'],
                num_stacked_layers=hyperparameters['num_layers']
            ).to(device)

            ### Optimizer, Criterion
            optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters['lr'])
            criterion = nn.MSELoss()

            validation_loss, model = train_model(
                model=model,
                train_loader=train_loader,
                test_loader=test_loader,
                optimizer=optimizer,
                criterion=criterion,
                device=device)
            
            # save loss for each run
            validation_losses.append(validation_loss)
        
        ### Save results to dict
        feature_acronym = ''.join([feature[0] for feature in features])
        RESULTS[f'{feature_acronym}_lag{hyperparameters["lag"]}_lr{hyperparameters["lr"]}_bs{hyperparameters["batch_size"]}_hs{hyperparameters["hidden_size"]}_nl{hyperparameters["num_layers"]}'] = min(validation_losses) # only save min loss
    
### Save results to json
print('Saving results to json...')
with open(f'./results/LSTM_benchmark_results_{datetime.now().strftime("%Y_%m_%d_%H%M%S")}.json', 'w') as json_file:
    json.dump(RESULTS, json_file, indent=4)
print('Results saved!')

print('#'*50)
print('#'*50)
print('#'*50)
print('TRAINING FINISHED')
print('#'*50)
print('#'*50)
print('#'*50)

## 3 Evaluate Results

In [ ]:
RESULTS

In [ ]:
best_params = min(RESULTS, key=RESULTS.get)
best_params

In [ ]:
RESULTS[best_params]

In [ ]:
print('CV_lag7_lr0.001_bs32_hs12_nl1')